<a href="https://colab.research.google.com/github/nssn96/ML_GestureRecognition/blob/main/ML_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 4 - Gesture Recognition using HMM(Hidden Markov models)


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [262]:
#install hmmlearn using pip3
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [263]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import pickle
from hmmlearn import hmm

In [264]:
#References Used
#https://hmmlearn.readthedocs.io/en/stable/tutorial.html#available-models
#https://datagy.io/pandas-groupby/

In [ ]:
#Importing the dataset
dataset = pd.read_csv('Original_Dataset.csv')
print(dataset)

In [ ]:
dataset.columns

In [267]:
''' Since we may not need all of that information and will probably perform well
using only the hand centroids, we will be dropping some of the columns from the dataset. We will take only take the x and y values
of the hands. '''
#Reference Used
#https://datagy.io/pandas-groupby/
#Normalize each sequence and dropping some column data
def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

dropped_columns = []
for column in dataset.columns:
    if str(column).find('LF') != -1 or  str(column).find('RF') != -1 or\
            str(column).find('LD') != -1 or str(column).find('RD') != -1:
        dropped_columns.append(column)
    elif column != 'Gesture':
        dataset[column] = normalize(dataset[column])
dataset = dataset.drop(dropped_columns,axis=1)
gestures = set(dataset.Gesture)
#print(dataset)
print(gestures)   ##this is of set type

{'Press', 'MoveRight', 'MoveUp', 'MoveDown', 'MoveLeft', 'ZoomIn', 'ZoomOut'}


In [268]:
# splitting the datset into train and test set--> I am splitting 90% train and 10% test set
basic_template = [0,0,0,0,0,0,0]  # here the size is 7 because, we have seven different classes
x_train = basic_template.copy()
x_test = basic_template.copy()
y_train = basic_template.copy()
y_test = basic_template.copy()
groupby_gesture = dataset.groupby(dataset.Gesture)
print(type(groupby_gesture))
for i, gesture in enumerate(gestures):
  gesture_data = groupby_gesture.get_group(gesture)
  x_train[i],x_test[i],y_train[i],y_test[i] = train_test_split(gesture_data.iloc[: , :120],gesture_data['Gesture'],test_size=0.2,stratify = gesture_data['Gesture'])
print('shape of train data:', np.array(x_train).shape)
print('shape of test data:',np.array(x_test).shape )

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
shape of train data: (7, 40, 120)
shape of test data: (7, 10, 120)


In [269]:
np.array(x_test).shape[1]

10

In [ ]:
print(dataset)

In [271]:
#Reference
#https://hmmlearn.readthedocs.io/en/stable/tutorial.html#available-models
#https://www.programcreek.com/python/example/105057/hmmlearn.hmm.GaussianHMM

#Building and training the model using  hmmlearn, an HMM library for Python

def model_build(components=7, iterations=100):
  mod = [0]*7
  r = np.random.RandomState(0)
  for i in range(len(gestures)):
    mod[i] = hmm.GaussianHMM(n_components=components, covariance_type="spherical",random_state=r,n_iter=iterations)
    mod[i].fit(np.array(x_train[i]).reshape(-1, 4), lengths=([30]*40))
  with open('models_7_HMM', 'wb') as files:
    pickle.dump(mod, files)
  acc = model_test(mod)
  return acc

In [ ]:
for i in range(len(list(gestures))):
  print(list(gestures)[i])

In [273]:
#This is the function that we use for testing the model
accuracy=[]
def model_test(mod):
  accuracy=[]
  test_size = np.array(x_test).shape[1]
  for i in range(len(gestures)):
    pred_correct=0
    #print("Gesture : ",list(gestures)[i])
    print("===================Gesture being tested : ",np.array(y_test)[i][0],"===========================")
    for j in range(test_size):
      #print("============Gesture being tested : ",np.array(y_test)[j][0])
      print("Sample ",j+1)
      m_score,m_index=0,0
      for k in range(len(mod)):
        score = mod[k].score(np.array(x_test)[i][j].reshape(30,4))
        m_score = max(m_score,score)
        if m_score == score:
          m_index = k
      print('The model_num ', m_index+1,'with highest model score : ',m_score)
      print("The Predicted Gesture : ",list(gestures)[m_index])
      print("    ")
      if m_index==i:
        pred_correct+=1
    accuracy.append(pred_correct/test_size)
  return accuracy



In [ ]:
# print('Components & 100 iterations')
# value = model_build(1)

# print('For 1 component & 100 iterations- The accuracy is')
# for i in range(len(set(dataset.Gesture))):
#     print('Model',i+1,'accuracy:', accuracy[i])

component_value=1
iterations=100
for i in range(len(gestures)):
  print('For Component ',component_value,' and ',iterations,' iterations')
  acc = model_build(component_value)
  #print(acc)
  component_value+=1
  #print("Gesture : ",list(gestures)[i])
  for i in range(len(gestures)):
    print('model',i+1,'accuracy value --> ', acc[i]*100,'%')
  print("    ")



In [285]:
#Finding the Best Configuration
#For the tuning the hyperparameter, I will be using random comp values in a list and check for which part the accuracy is better
#Iterations---> I have multiplied the component number * 10
diff_comp = [8,9,10,12,14]
for comp in diff_comp:
  iterations = comp*10
  for i in range(len(gestures)):
    print('For Component ',comp,' and ',iterations,' iterations')
    acc = model_build(comp,iterations)
    #print(acc)
    #component_value+=1
    #print("Gesture : ",list(gestures)[i])
    for i in range(len(gestures)):
      print('model',i+1,'accuracy value --> ', acc[i]*100,'%')



Streaming output truncated to the last 5000 lines.
The model_num  1 with highest model score :  416.7610807822939
The Predicted Gesture :  Press
    
Sample  5
The model_num  1 with highest model score :  411.7382117140343
The Predicted Gesture :  Press
    
Sample  6
The model_num  1 with highest model score :  404.7734487654383
The Predicted Gesture :  Press
    
Sample  7
The model_num  1 with highest model score :  420.2923503911808
The Predicted Gesture :  Press
    
Sample  8
The model_num  1 with highest model score :  416.5034589979433
The Predicted Gesture :  Press
    
Sample  9
The model_num  1 with highest model score :  385.74120037518935
The Predicted Gesture :  Press
    
Sample  10
The model_num  1 with highest model score :  394.1577968892782
The Predicted Gesture :  Press
    
===================Gesture being tested :  MoveRight ===========================
Sample  1
The model_num  2 with highest model score :  397.21757823830717
The Predicted Gesture :  MoveRight
    

In [ ]:
from pandas.core.algorithms import mode
#References
#https://hmmlearn.readthedocs.io/en/latest/auto_examples/plot_hmm_sampling_and_decoding.html#sphx-glr-auto-examples-plot-hmm-sampling-and-decoding-py
#https://hmmlearn.readthedocs.io/en/0.2.0/auto_examples/plot_hmm_sampling.html
#Sampling from the HMM

with open('models_7_HMM' , 'rb') as files:
    models = pickle.load(files)



for i in range(len(gestures)):
    fig = plt.figure(figsize=[16, 18])
    ax = fig.add_subplot(2, 2, 1)
    obs = fig.add_subplot(2, 2, 2)
    ax.plot(models[i].sample(30)[0].reshape(), color='b', linestyle="-")
    obs.plot(np.array(x_train)[i][0].reshape(30,4), color='g', linestyle="-")
    sample_label = 'Model '+str(i+1)+':'+list(gestures)[i]+' Sample Data'
    training_label = 'Model '+str(i+1)+':'+list(gestures)[i]+' Training Data'
    ax.title.set_text(sample_label)
    obs.title.set_text(training_label)



